In [12]:
H0=  0xc1059ed8
H1 = 0x367cd507
H2 = 0x3070dd17
H3 = 0xf70e5939
H4 = 0xffc00b31
H5 = 0x68581511
H6 = 0x64f98fa7
H7 = 0xbefa4fa4

In [13]:
K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
    0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
    0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
    0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
    0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
    0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
    0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
    0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
    0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]


In [14]:
def preprocessing_input(input):
    n=len(input)
    k= (448-(1+n*8))%512
    print('1'+ k*'0'+ number_to_binary(n*8))
    print(len('1'+ k*'0'+ number_to_binary(n*8)))
def number_to_binary(n):
    """Convert an integer to a 64-bit binary string."""
    if not isinstance(n, int):
        raise ValueError("Input must be an integer.")
    return bin(n)[2:].zfill(64)  # Ensures exactly 64 bits

def preprocessing_input(input):
    """Prepares input for SHA-1 by adding padding and length encoding."""
    input_bytes = input.encode('utf-8')  # Convert to bytes
    n = len(input_bytes)  # Length in bytes
    # n=len(input)
    # print(n)
    message_bits = ''.join(format(byte, '08b') for byte in input_bytes)  # Convert each byte to 8-bit binary
    message_bits += '1'  # Append single '1' bit
    # Compute k (number of zero bits required)
    k = (448 - ((n * 8 + 1) % 512)) % 512
    message_bits += '0' * k  # Append k zero bits
    # Append original length in 64-bit binary
    message_bits += number_to_binary(n * 8)
    return message_bits
def right_rotate(n, b):
    """Right rotate a 32-bit integer n by b bits."""
    return ((n >> b) | (n << (32 - b))) & 0xFFFFFFFF  # Ensure 32-bit result    
def SHR(x,n):
    return x>>n

In [15]:
def summation_0_func(x):
    # print(f"Length of x in bits: {x.bit_length()}")  # Print bit length of x
    return right_rotate(x,2) ^ right_rotate(x,13) ^ right_rotate(x,22)
def summation_1_func(x):
    return right_rotate(x,6) ^ right_rotate(x,11) ^ right_rotate(x,25)
def sigma_0_func(x):
    return right_rotate(x,7) ^ right_rotate(x,18) ^ SHR(x,3)
def sigma_1_func(x):
    return right_rotate(x,17) ^ right_rotate(x,19) ^ SHR(x,10)
def ch(x,y,z):
     return (x & y) ^ ((~x) & z)
def maj(x,y,z):
    return (x & y) ^ (y & z) ^ (x & z)

In [16]:
def message_schedule_16_blocks(padded_message_block):
    """
    Extracts W₀ to W₁₅ from a 512-bit padded message block and converts to hex.
    Assumes input is a string of '0' and '1' (binary representation).
    """
    W = []
    for i in range(16):  # 16 words of 32 bits each
        word = padded_message_block[i * 32: (i + 1) * 32]  # Extract 32-bit chunk
        word_int = int(word, 2)  # Convert binary string to integer
        W.append(word_int)
    return W

In [17]:
def words_17_to_64(W):
    """
    Expands W₀ to W₆₃ using:
    Wₜ = (σ₁(Wₜ₋₂) + Wₜ₋₇ + σ₀(Wₜ₋₁₅) + Wₜ₋₁₆) mod 2³²
    """
    for i in range(16, 64):
        new_word = (sigma_1_func(W[i-2]) + W[i-7] + sigma_0_func(W[i-15]) + W[i-16]) & 0xFFFFFFFF
        W.append(new_word)  # Ensure it's 32-bit
    return W

In [18]:
def Nth_block_output(input,h0_temp,h1_temp,h2_temp,h3_temp,h4_temp,h5_temp,h6_temp,h7_temp):
  a, b, c, d, e, f,g,h = h0_temp, h1_temp, h2_temp, h3_temp, h4_temp,h5_temp, h6_temp,h7_temp
  W=words_17_to_64(message_schedule_16_blocks(input))
  for i in range(64):
    T1=(h + summation_1_func(e) + ch(e,f,g) + K[i] + W[i]) & 0xFFFFFFFF 
    T2= (summation_0_func(a) + maj(a,b,c)) & 0xFFFFFFFF 
    h=g
    g=f
    f=e
    e=(d+ T1) & 0xFFFFFFFF
    d=c
    c=b
    b=a
    a= (T1+ T2) & 0xFFFFFFFF 

  h0_temp = (h0_temp + a) & 0xFFFFFFFF
  h1_temp = (h1_temp + b) & 0xFFFFFFFF
  h2_temp = (h2_temp + c) & 0xFFFFFFFF   
  h3_temp = (h3_temp + d) & 0xFFFFFFFF
  h4_temp = (h4_temp + e) & 0xFFFFFFFF
  h5_temp = (h5_temp + f) & 0xFFFFFFFF
  h6_temp = (h6_temp + g) & 0xFFFFFFFF
  h7_temp = (h7_temp + h) & 0xFFFFFFFF
  return h0_temp,h1_temp,h2_temp,h3_temp,h4_temp,h5_temp,h6_temp,h7_temp

In [42]:
def generalized_SHA224(input):
    h0=  0xc1059ed8
    h1 = 0x367cd507
    h2 = 0x3070dd17
    h3 = 0xf70e5939
    h4 = 0xffc00b31
    h5 = 0x68581511
    h6 = 0x64f98fa7
    h7 = 0xbefa4fa4
    h0_temp, h1_temp, h2_temp, h3_temp, h4_temp , h5_temp,h6_temp,h7_temp= h0, h1, h2, h3, h4 ,h5,h6,h7 
    length_processed_text = len(preprocessing_input(input))  
    input_temp=preprocessing_input(input); 
    for i in range(length_processed_text // 512):
        if i == 0:
            h0_temp, h1_temp, h2_temp, h3_temp, h4_temp,h5_temp,h6_temp,h7_temp = Nth_block_output(input_temp[0:512], h0, h1, h2, h3, h4,h5,h6,h7)
        else:    
            h0_temp, h1_temp, h2_temp, h3_temp, h4_temp,h5_temp,h6_temp,h7_temp = Nth_block_output(input_temp[i * 512:(i + 1) * 512], 
                                                                           h0_temp, h1_temp, h2_temp, h3_temp, h4_temp,h5_temp,h6_temp,h7_temp)
        
        # 🔹 Print intermediate values in hex for debugging
        # print(f"After block {i+1}:")
        # print(f"h0 = {h0_temp:08x}")
        # print(f"h1 = {h1_temp:08x}")
        # print(f"h2 = {h2_temp:08x}")
        # print(f"h3 = {h3_temp:08x}")
        # print(f"h4 = {h4_temp:08x}")
        # print("-" * 50)  # Separator for readability

    # Convert each temp value to a zero-padded 8-character hex string before concatenating
    return ''.join([
        f"{h0_temp:08x}", 
        f"{h1_temp:08x}", 
        f"{h2_temp:08x}", 
        f"{h3_temp:08x}", 
        f"{h4_temp:08x}",
        f"{h5_temp:08x}",
        f"{h6_temp:08x}"
    ])
# print(generalized_SHA224("hi i am here how are you\nhi i am here how are you"))
input_text=input("enter plain text: ")
print("Message Digest(SHA-224): ",generalized_SHA224(input_text))
# import time 
# t1=time.perf_counter()
#print(generalized_SHA256("a"*1000000))
# t2=time.perf_counter();
# print(t2-t1)

enter plain text:  I LOVE INDIA93f25d


Message Digest(SHA-224):  269655d21e7ebed9c6703c664dc47456cfb24791a7e6165ba73d4950


In [ ]:
import os

def generate_hex_salt(length=6):
    """Generate a random hex string of given length."""
    return os.urandom(length // 2).hex()[:length]  # Generate bytes & convert to hex

salt = generate_hex_salt(6)
print("Generated Salt:", salt)


Generated Salt: c7ef36


In [80]:
def generalized_SHA224_salt(input_text):
    salt = generate_hex_salt(6)  # Generate salt
    input_text = input_text + salt  # Append salt before hashing

    # SHA-224 initial hash values (as per NIST FIPS 180-4)
    h0 = 0xc1059ed8
    h1 = 0x367cd507
    h2 = 0x3070dd17
    h3 = 0xf70e5939
    h4 = 0xffc00b31
    h5 = 0x68581511
    h6 = 0x64f98fa7
    h7 = 0xbefa4fa4

    # Temporary variables
    h0_temp, h1_temp, h2_temp, h3_temp, h4_temp, h5_temp, h6_temp, h7_temp = h0, h1, h2, h3, h4, h5, h6, h7

    # Preprocessing input
    input_temp = preprocessing_input(input_text)
    length_processed_text = len(input_temp)

    # Process each 512-bit block
    for i in range(length_processed_text // 512):
        block = input_temp[i * 512:(i + 1) * 512]
        if i == 0:
            h0_temp, h1_temp, h2_temp, h3_temp, h4_temp, h5_temp, h6_temp, h7_temp = Nth_block_output(
                block, h0, h1, h2, h3, h4, h5, h6, h7)
        else:
            h0_temp, h1_temp, h2_temp, h3_temp, h4_temp, h5_temp, h6_temp, h7_temp = Nth_block_output(
                block, h0_temp, h1_temp, h2_temp, h3_temp, h4_temp, h5_temp, h6_temp, h7_temp)

    # Convert hash values to hexadecimal format
    raw_hash = ''.join([
        f"{h0_temp:08x}",
        f"{h1_temp:08x}",
        f"{h2_temp:08x}",
        f"{h3_temp:08x}",
        f"{h4_temp:08x}",
        f"{h5_temp:08x}",
        f"{h6_temp:08x}"
    ])

    # Split salt into three parts
    salt1, salt2, salt3 = salt[:2], salt[2:4], salt[4:]

    # Insert salt at different positions
    mixed_hash = salt1 + raw_hash[:14] + salt2 + raw_hash[14:28] + salt3 + raw_hash[28:]

    return mixed_hash, salt  # Return mixed hash & salt for verification

# Get user input and hash the message
input_text = input("Enter plain text: ")
arr=[]
for i in range(0,50):
  hashed_output, used_salt = generalized_SHA224_salt(input_text)
  arr.append(hashed_output)
  print("Message Digest (SHA-224 with Salt Mixing):", hashed_output,len(hashed_output))
  print("Used Salt:", used_salt)

Enter plain text:  I AM LOVE , I AM WEALTHY , I AM ABUNDANT , I AM PEACEFUL , I AM HEALTHY


Message Digest (SHA-224 with Salt Mixing): 6853b04dab49c78ff4fa9a2e5823e578e001aef7c2d742277ccec019710ba1 62
Used Salt: 68f4e0
Message Digest (SHA-224 with Salt Mixing): e9c90b30c1641c02f010d2145e41fed181178da257a83abf2dd8953d9a1c03 62
Used Salt: e9f081
Message Digest (SHA-224 with Salt Mixing): 9cdfc800db72d742c1f32456cb2b703037f4e392e96ebba9edc88f71232314 62
Used Salt: 9cc137
Message Digest (SHA-224 with Salt Mixing): cf75ab8a6c04e6e99848c07687263e8f51f17e06d62f241439b994c283163c 62
Used Salt: cf9851
Message Digest (SHA-224 with Salt Mixing): 66f7f3cb4bf7e32257125c4a98f1651dbf91a33c675c61885d34feb5d4385f 62
Used Salt: 6657bf
Message Digest (SHA-224 with Salt Mixing): a689f5e33a51be141ea31ec8972bce3a8b3a5d46889783c56eb07426ffe972 62
Used Salt: a61e8b
Message Digest (SHA-224 with Salt Mixing): 9143ff84a3aa0f7cf0319bee72d4273cc103082deabfc08a5c93065df081bf 62
Used Salt: 91f0c1
Message Digest (SHA-224 with Salt Mixing): 39171835494b670fc73e1d3886ce8a10ba457a663ef8f5a3e48fd5cfefdf20 62
Us

In [81]:
print(arr)

['6853b04dab49c78ff4fa9a2e5823e578e001aef7c2d742277ccec019710ba1', 'e9c90b30c1641c02f010d2145e41fed181178da257a83abf2dd8953d9a1c03', '9cdfc800db72d742c1f32456cb2b703037f4e392e96ebba9edc88f71232314', 'cf75ab8a6c04e6e99848c07687263e8f51f17e06d62f241439b994c283163c', '66f7f3cb4bf7e32257125c4a98f1651dbf91a33c675c61885d34feb5d4385f', 'a689f5e33a51be141ea31ec8972bce3a8b3a5d46889783c56eb07426ffe972', '9143ff84a3aa0f7cf0319bee72d4273cc103082deabfc08a5c93065df081bf', '39171835494b670fc73e1d3886ce8a10ba457a663ef8f5a3e48fd5cfefdf20', '6f6866ef18a0bfa52d78f2a82c075c03fde84a87939acad106cc897caec190', '7a40ea5a4b51a7c73ba9bcdf528b41e3c010d6071d3c26151b7af547d8dc48', 'ab084cdcb4095eee118d1228567142ad8a1f81b12f7acf5ebd30de4f66d63f', '0dc116126c3ddf041eb11724de953eb3a2a5840d7426448508c4ac3411fc89', '415aac089d6a5e7fd844ad79087008d50638a454adfcdb637f001a06f21c83', 'ec7b136b11eefaf99b798663740f74d5b81520fb130217e1054ac29111db2e', 'be50f4417fad0048df284be183fc6d6cfa5d066bd0fa282460e339264fb256', 'a248264a

In [ ]:
def extract_salt_from_hash(mixed_hash):
    """Extracts the salt from the mixed SHA-224 hash."""
    salt1 = mixed_hash[:2]      # First 2 characters (inserted at the beginning)
    salt2 = mixed_hash[16:18]   # Middle 2 characters (inserted after 14 hex chars)
    salt3 = mixed_hash[32:34]   # Last 2 characters (inserted after 28 hex chars)
    return salt1 + salt2 + salt3  # Reconstruct the original 6-character salt

mixed_hash = "84c6e7634063c4d6e8041c0d5a2e0508a2a3faeaac7fb9ec4e6448d9e7004c62"
extracted_salt = extract_salt_from_hash(mixed_hash)
print("Extracted Salt:", extracted_salt)

Extracted Salt: 84e8a2


In [ ]:
def remove_salt_from_hash(mixed_hash):
    """Removes the salt from the mixed SHA-224 hash and reconstructs the original hash."""
    # Extract parts of the hash without salt
    hash_without_salt = (
        mixed_hash[2:16] +  # Part before salt2 (skip first 2 chars)
        mixed_hash[18:32] +  # Part before salt3 (skip salt2)
        mixed_hash[34:62]  # Remaining part after salt3
    )
    return hash_without_salt  # Return clean hash

mixed_hash = "84c6e7634063c4d6e8041c0d5a2e0508a2a3faeaac7fb9ec4e6448d9e7004c62"
clean_hash = remove_salt_from_hash(mixed_hash)
print("Original SHA-224 Hash (Without Salt):", clean_hash)


Original SHA-224 Hash (Without Salt): c6e7634063c4d6041c0d5a2e0508a3faeaac7fb9ec4e6448d9e7004c


In [70]:
print(generalized_SHA224("HI I AM PRASANNA"+extracted_salt)==clean_hash)

True


In [77]:
def compare_hash(input,hash_value):
    clean_hash=remove_salt_from_hash(hash_value)
    extracted_salt= extract_salt_from_hash(hash_value)
    return generalized_SHA224(input+extracted_salt)==clean_hash

In [83]:
for i1 in range(len(arr)):
    if(compare_hash("I AM LOVE , I AM WEALTHY , I AM ABUNDANT , I AM PEACEFUL , I AM HEALTHY",arr[i1])):
        print("case ",i1+1," passed, match found ✅")
    else:
        print("match not found")

case  1  passed, match found ✅
case  2  passed, match found ✅
case  3  passed, match found ✅
case  4  passed, match found ✅
case  5  passed, match found ✅
case  6  passed, match found ✅
case  7  passed, match found ✅
case  8  passed, match found ✅
case  9  passed, match found ✅
case  10  passed, match found ✅
case  11  passed, match found ✅
case  12  passed, match found ✅
case  13  passed, match found ✅
case  14  passed, match found ✅
case  15  passed, match found ✅
case  16  passed, match found ✅
case  17  passed, match found ✅
case  18  passed, match found ✅
case  19  passed, match found ✅
case  20  passed, match found ✅
case  21  passed, match found ✅
case  22  passed, match found ✅
case  23  passed, match found ✅
case  24  passed, match found ✅
case  25  passed, match found ✅
case  26  passed, match found ✅
case  27  passed, match found ✅
case  28  passed, match found ✅
case  29  passed, match found ✅
case  30  passed, match found ✅
case  31  passed, match found ✅
case  32  passed,

In [84]:
import hashlib
import random
import string
import time

In [ ]:
def generate_random_string(length):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

# Function to compare SHA-224 implementations
def test_sha224_accuracy_and_speed():
    num_tests = 3000  # Number of test cases
    start_length = 10  # Initial string length
    total_tests_passed = 0
    total_time_custom = 0
    total_time_hashlib = 0

    print("Running SHA-224 Tests...\n")

    for i in range(num_tests):
        test_length = start_length + i  # Increase input length with each test
        test_input = generate_random_string(test_length)  # Generate random input

        # Measure time for custom SHA-224
        start_time_custom = time.perf_counter()
        custom_sha224 = generalized_SHA224(test_input)
        end_time_custom = time.perf_counter()
        total_time_custom += (end_time_custom - start_time_custom)

        # Measure time for hashlib SHA-224
        start_time_hashlib = time.perf_counter()
        hashlib_sha224 = hashlib.sha224(test_input.encode()).hexdigest()
        end_time_hashlib = time.perf_counter()
        total_time_hashlib += (end_time_hashlib - start_time_hashlib)

        # Check if hashes match
        if custom_sha224 == hashlib_sha224:
            total_tests_passed += 1
        else:
            print(f"❌ Mismatch at test {i + 1}:")
            print(f"Input: {test_input}")
            print(f"Custom SHA-224:  {custom_sha224}")
            print(f"Hashlib SHA-224: {hashlib_sha224}")
            print("----")

    print("\n✅ Test Results:")
    print(f"Total tests passed: {total_tests_passed} / {num_tests}")
    print(f"Total time for Custom SHA-224: {total_time_custom:.4f} seconds")
    print(f"Total time for Hashlib SHA-224: {total_time_hashlib:.4f} seconds")
    print(f"Custom SHA-224 is {(total_time_custom / total_time_hashlib):.2f} times slower than hashlib.")

test_sha224_accuracy_and_speed()

Running SHA-224 Tests...


✅ Test Results:
Total tests passed: 3000 / 3000
Total time for Custom SHA-224: 35.0636 seconds
Total time for Hashlib SHA-224: 0.0576 seconds
Custom SHA-224 is 609.11 times slower than hashlib.


In [ ]:
import hashlib
import random
import string

# Function to generate a 6-character salt with only letters (A-Z, a-z)
def generate_salt(length=6):
    """Generate a random salt containing only letters (no numbers)."""
    return ''.join(random.choices(string.ascii_letters, k=length))

# Function to find the first three numerical digits in a given hash
def find_nearest_digits(hash_str):
    """Finds the first three numerical digits in the hash and returns their sorted positions."""
    positions = [i for i, c in enumerate(hash_str) if c.isdigit()]
    return sorted(positions[:3])  # Take first 3 digits and sort

# Function to hash input + salt and embed the salt dynamically
def sha224_with_dynamic_salt(input_text):
    """Hashes (input + salt), finds digit positions, and inserts a salt dynamically."""
    salt = generate_salt(6)  # Generate a random 6-letter salt
    temp_hash = hashlib.sha224((salt + input_text).encode()).hexdigest()  # Compute hash of (salt + input)
    
    digit_positions = find_nearest_digits(temp_hash)  # Get first 3 digit positions
    if len(digit_positions) < 3:
        raise ValueError("SHA-224 hash did not contain enough digits. Try different input.")

    # Convert hash to list (mutable) for inserting salt
    salted_hash = list(temp_hash)

    # Insert salt at the identified positions
    salted_hash.insert(digit_positions[0], salt[:2])  # First 2 letters
    digit_positions[1] += 2  # Adjust position after first insert
    salted_hash.insert(digit_positions[1], salt[2:4])  # Middle 2 letters
    digit_positions[2] += 4  # Adjust position after second insert
    salted_hash.insert(digit_positions[2], salt[4:6])  # Last 2 letters

    return ''.join(salted_hash), salt  # Return salted hash and salt

# Function to extract salt and recover the original hash
def extract_salt_and_original_hash(salted_hash):
    """Extracts the salt and restores the original hash by removing it."""
    digit_positions = find_nearest_digits(salted_hash)  # Get digit positions
    if len(digit_positions) < 3:
        raise ValueError("Salted hash does not contain enough digits to extract salt.")

    # Convert salted hash to list for removing salt
    hash_list = list(salted_hash)

    # Extract salt using digit positions
    extracted_salt = hash_list[digit_positions[0]] + hash_list[digit_positions[0]+1] + \
                     hash_list[digit_positions[1]+2] + hash_list[digit_positions[1]+3] + \
                     hash_list[digit_positions[2]+4] + hash_list[digit_positions[2]+5]

    # Remove salt from hash
    del hash_list[digit_positions[2]+4:digit_positions[2]+6]
    del hash_list[digit_positions[1]+2:digit_positions[1]+4]
    del hash_list[digit_positions[0]:digit_positions[0]+2]

    return extracted_salt, ''.join(hash_list)

# Function to verify if a given input matches the stored salted hash
def verify_sha224_with_salt(input_text, stored_salted_hash):
    """Extracts salt, reconstructs original hash, and checks validity."""
    extracted_salt, extracted_hash = extract_salt_and_original_hash(stored_salted_hash)
    
    # Compute hash of (salt + input)
    computed_hash = hashlib.sha224((extracted_salt + input_text).encode()).hexdigest()

    print(f"Extracted Salt: {extracted_salt}")
    print(f"Computed Hash:  {computed_hash}")
    print(f"Extracted Hash: {extracted_hash}")

    return computed_hash == extracted_hash  # Compare hashes

input_text = "HELLO WORLD"

# Hash with dynamic salt insertion
salted_hash, salt = sha224_with_dynamic_salt(input_text)
print("Salted Hash:", salted_hash)
print("Original Salt:", salt)


# Verification
is_valid = verify_sha224_with_salt(input_text, salted_hash)
print("Is Hash Valid?", is_valid)


Salted Hash: LI84Ug66pxe2143373349fb8735baee823fd3924dc4c1b3b8c0103bf6dbcfc
Original Salt: LIUgpx
Extracted Salt: 84g6e2
Computed Hash:  328ce219ce4b1482c289b054455b8dd0406b1b8f0a43d59e1ce3d330
Extracted Hash: LIU6px143373349fb8735baee823fd3924dc4c1b3b8c0103bf6dbcfc
Is Hash Valid? False
